In [24]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.fft as fft
import seaborn as sns
sns.set()

from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tqdm.notebook import tqdm

import sys
sys.path.append("/home/evangelos/workspace/Channel_Charting/")
from tools import utils
from src.models import supervised_classifier, supervised_regressor
from src.utils.data_preparation import SupervisedDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# load datasets
(raw_taps, raw_phi, raw_theta, raw_rx_positions, raw_tx_positions) = utils.load_datasets()

In [26]:
#fourier transform and undersample taps
raw_freq_taps = fft.fft(raw_taps , workers=-1)[:,:,::2]

In [27]:
mean_freq = np.mean(raw_freq_taps, axis=(1,2))
mu = np.mean(mean_freq)
std = np.std(mean_freq)
idcs = ~(np.abs(mean_freq-mu) >3*std)

In [28]:
def preprocessing(data, first_data, rx_positions=None, padding=True, reduce=False, keep_idces=idcs):
    data = data[idcs]
    if rx_positions:
        data = utils.drop_top_right(data, rx_positions)
    data = utils.standarize(data)
    data = utils.fillna(data)
    if padding:
        data = utils.zero_padding_as(data, first_data)
    #data = utils.take_norm(data)
    if reduce:
        data = reduce_to_mean_std(data)
    
    return data

def reduce_to_mean_std(x):
    return np.stack([np.mean(x,axis=1), 
                          np.std(x,axis=1)], axis=1)

In [29]:
taps = preprocessing(raw_freq_taps, raw_freq_taps)
taps = np.hstack([np.real(taps), np.imag(taps)])

In [30]:
phi = preprocessing(raw_phi, taps)

In [31]:
theta = preprocessing(raw_theta, taps)

In [32]:
y  = preprocessing(raw_rx_positions, taps, padding=False, reduce=False)[:,:2]

In [33]:
X = np.hstack([taps, phi[:-10], theta[:-10]])

In [ ]:
#assign labels to certain areas of the map using kmeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=8) 
km = km.fit(y)
labels = km.predict(y)

In [ ]:
sns.scatterplot(y[:,0], y[:,1], hue=labels, alpha=0.3)

In [14]:
#train test split
train_X, test_X ,train_y, test_y, train_labels, test_labels = train_test_split(X, y, labels, train_size=0.1, test_size=0.1)
train_DS = SupervisedDataset(train_X, train_labels)
test_DS = SupervisedDataset(test_X, test_labels)

In [15]:
train_loader = DataLoader(train_DS, batch_size=32)
test_loader = DataLoader(test_DS)

In [16]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

model = supervised_classifier.Classifier(channels=train_DS.channels(), nb_labels=8)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [17]:
best_val_loss = 9999
count = 0
for epoch in tqdm(range(100)):
    loss = supervised_classifier.train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = supervised_classifier.test(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}: Training Loss {loss}, Validation Loss {val_loss}, Validation Accuracy {val_acc}")
    
    if best_val_loss < val_loss:
        count += 1
    else:
        best_val_loss = val_loss
    if count > 5:
        break

Epoch 1: Training Loss 0.06475594766516435, Validation Loss 1.9161578823721368, Validation Accuracy 0.24537037037037038
Epoch 2: Training Loss 0.056171975838510614, Validation Loss 1.6578630699606782, Validation Accuracy 0.3413299663299663
Epoch 3: Training Loss 0.04940608667072497, Validation Loss 1.5400295177034042, Validation Accuracy 0.3985690235690236
Epoch 4: Training Loss 0.04634516999596044, Validation Loss 1.451141759601461, Validation Accuracy 0.43392255892255893
Epoch 5: Training Loss 0.04455153256968448, Validation Loss 1.4191720065586693, Validation Accuracy 0.4532828282828283
Epoch 6: Training Loss 0.04326697773682444, Validation Loss 1.368351403358504, Validation Accuracy 0.49074074074074076
Epoch 7: Training Loss 0.04142431547767238, Validation Loss 1.335725130479711, Validation Accuracy 0.5058922558922558
Epoch 8: Training Loss 0.041026750464188425, Validation Loss 1.3363925233198009, Validation Accuracy 0.5046296296296297
Epoch 9: Training Loss 0.04010982116899992, Va

In [1]:
yhats = model(test_DS[:][0]).detach()
predictions = yhats.argmax(dim=1)
plot = sns.scatterplot(test_y[:,0], test_y[:,1], hue=predictions)

NameError: name 'model' is not defined

In [2]:
print(classification_report(predictions, test_labels))

NameError: name 'classification_report' is not defined

In [ ]:
sns.scatterplot(test_y[:,0], test_y[:,1], hue=test_labels)

In [ ]:
sns.heatmap(confusion_matrix(predictions, test_labels))

In [ ]:
accuracy_score(predictions, test_labels)

In [ ]:
report = classification_report(predictions, test_labels, output_dict=True)

In [ ]:
report